In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, date, timedelta

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from scipy.cluster import hierarchy

import sqlite3
from sqlalchemy               import create_engine
from sqlalchemy.pool          import NullPool

# from sklearn.decomposition import PCA
# from umap.umap_ import UMAP
# from sklearn.manifold import TSNE

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Pappermill Checkings

papermill_record = 6
papermill_hostname = 'machine_name'

In [3]:
PATH = '/Users/Alysson/Documents/Projects/E-Commerce-Clusterization/data/ecommerce.csv'
data_raw = pd.read_csv(PATH, encoding='iso-8859-1')
data = data_raw.copy()

In [4]:
data = data.drop('Unnamed: 8', axis=1)
data = data.dropna(subset=['CustomerID'])

In [5]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format='%d-%b-%y')
data['CustomerID'] = data['CustomerID'].astype(int)

data['Total'] = data['Quantity'] * data['UnitPrice']

In [6]:
sum_transactions_per_client=data[['CustomerID','Total','Quantity']].groupby('CustomerID').agg({'Total':np.sum,
                                                                 'Quantity':np.sum,                               
                                                                 #'CustomerID':np.unique                                                                                    
                                                                 }).reset_index()

In [7]:
#Customers who do not have a positive purchase balance or who owe the company (due to the temporal cut of the database) will be excluded

bad_clients = sum_transactions_per_client.loc[(sum_transactions_per_client['Total'] <= 0.5) | (sum_transactions_per_client['Quantity'] <= 1)]

In [8]:
list_bad_clients=bad_clients['CustomerID'].tolist()
data = data[~data['CustomerID'].isin(list_bad_clients)]

In [9]:
data = data.loc[~(data['UnitPrice'] < 0.04)]

In [10]:
list_letter_stock=data[data.StockCode.str.contains("^[a-zA-Z]")].StockCode.value_counts().index.tolist()

In [11]:
data = data.loc[~data['StockCode'].isin(list_letter_stock)]

In [12]:
transactions = data.copy()

In [13]:
#Group InvoiceNumber, it contains sales and cancelations

transactions=data.groupby('InvoiceNo').agg( CustomerID = ('CustomerID', np.unique),
                                            InvoiceDate = ('InvoiceDate', np.unique),
                                            Total = ('Total', 'sum'),
                                            UniqueProducts = ('StockCode', 'nunique'), 
                                            Items = ('Quantity', 'sum'),
                                            Country = ('Country', np.unique),
                                            ProductsCode = ('StockCode', np.unique)).reset_index()

transactions['AvarageTicket']= round(transactions['Total']/transactions['UniqueProducts'],2) 
#len(transactions)

In [14]:
last_day = data.InvoiceDate.max() + dt.timedelta(days = 1)

transactions_per_customer = transactions.groupby('CustomerID').agg(
                                                      GrossRevenue = ('Total', np.sum),                                           
                                                      Recency = ('InvoiceDate', lambda x: ((last_day - x.max()).days)),             
                                                      Frequency = ('InvoiceNo', 'count'),             
                                                      Products = ('UniqueProducts', 'sum'), 
                                                      Items = ('Items', 'sum'),  
                                                      Country = ('Country', np.unique),
                                                      AvarageTicket = ('AvarageTicket', 'sum'))            
                                                      #Products = ('StockCode', np.unique),
                                                                   
#transactions_per_customer['AvarageTicket']= round(transactions_per_customer['GrossRevenue'] / transactions_per_customer['Products'],2)

## Data Preparation

In [15]:
data_prep = transactions_per_customer.copy()

In [16]:
data_prep=data_prep.dropna()

In [17]:
categorical_features = ['Country']
numerical_features = ['GrossRevenue', 'Items', 'AvarageTicket', "Products", 'Frequency', "Recency"]

#data_prep["Country"] = [0 if i == 'Norway' else 1 for i in data_prep["Country"]]

default_features = data_prep.columns
selected_features = ['GrossRevenue','Recency','Frequency']

data_prep=data_prep[selected_features].copy() 

In [18]:
log_columns = data_prep[selected_features].skew().sort_values(ascending=False)
log_columns = log_columns.loc[log_columns > 0.75]
log_columns

GrossRevenue    21.526419
Frequency       11.401617
Recency          1.265656
dtype: float64

In [19]:
# The log transformations
for col in log_columns.index:
    data_prep[col] = np.log1p(data_prep[col])

In [20]:
#ss = StandardScaler()
#rs = RobustScaler()
#pt = PowerTransformer()

# MinmmaxScaler provides better resutls mainly because is robbust to outliers
mms = MinMaxScaler()

for col in data_prep[selected_features]:
    data_prep[col] = mms.fit_transform(data_prep[[col]]).squeeze()
    
X = data_prep[selected_features].copy() 

In [21]:
clusters_results = X.copy()

In [22]:
# # Dimensionality Reduction(to 2D) using technique UMAP 
# umap = UMAP(random_state=3456)
# umap_embedding = umap.fit_transform(X)

# # X,y axis representation for UMAP
# clusters_results['umap_x'] = umap_embedding[:,0]
# clusters_results['umap_y'] = umap_embedding[:,1]

In [23]:
# # Dimensionality Reduction(to 2D) using technique TSNE
# tsne = TSNE(n_components=2, init='pca', learning_rate='auto', n_jobs=-1, random_state=3456)
# tsne_embedding = tsne.fit_transform(X)

# # X,y axis representation using TSNE
# clusters_results['tsne_x'] = tsne_embedding[:,0]
# clusters_results['tsne_y'] = tsne_embedding[:,1]

## Model - Hierachical Cluster

In [24]:
k=10
#hc = AgglomerativeClustering(k, affinity='euclidean', linkage='complete', compute_full_tree=False)
hc = hierarchy.linkage(X, 'ward', metric='euclidean')
hc_labels = hierarchy.fcluster(hc, k, criterion='maxclust')
clusters_results["HierarchicalCluster"]=hc_labels

In [25]:
all_clusters=transactions_per_customer.copy()

In [26]:
all_clusters['HC'] = hc_labels

In [27]:
all_clusters[['GrossRevenue','Recency','Frequency','HC']].groupby('HC').mean().sort_values(by='GrossRevenue', ascending=False).head(20)

,GrossRevenue,Recency,Frequency
HC,,,
3,13793.701210,6.370107,25.982206
1,3713.345916,42.475884,10.581994
2,1678.157504,18.075829,5.451817
8,1529.239724,113.877880,4.467742
4,1522.371525,3.662757,5.351906
6,802.785489,39.525862,2.974138
9,641.665171,196.627376,2.110266
7,363.514452,60.165501,1.289044
5,298.804777,19.449045,1.512739


In [28]:
report = all_clusters[['GrossRevenue','Recency','Frequency','HC']].groupby('HC').agg( MonetarySum = ('GrossRevenue', 'sum'),
                                             MonetaryMean = ('GrossRevenue', 'mean'),
                                             Recency = ('Recency', 'mean'),
                                             Frequency = ('Frequency', 'mean'),
                                             Count = ('GrossRevenue', 'count')).reset_index()

report['Percentage'] = round((report['Count'] / report['Count'].sum()*100),2)
report=report.sort_values(by='MonetaryMean', ascending=False)

In [29]:
report['Cluster'] = ['Champion', #3
                     'Loyal Costumer', #4
                     'Potential Loyalist',#1
                     'Cannot Lose Them', #7 
                     'New Customers', #9
                     'Promising', #5
                     'About to Sleep',    #2               
                     'Need Atention', #10
                     'At Risk',   #8
                     'Hinernating']   #6

In [30]:
report.set_index('Cluster', inplace=True)
report.drop('HC', axis=1,inplace=True)

In [31]:
report

,MonetarySum,MonetaryMean,Recency,Frequency,Count,Percentage
Cluster,,,,,,
Champion,3876030.04,13793.701210,6.370107,25.982206,281,6.52
Loyal Costumer,1154850.58,3713.345916,42.475884,10.581994,311,7.21
Potential Loyalist,1062273.70,1678.157504,18.075829,5.451817,633,14.68
Cannot Lose Them,663690.04,1529.239724,113.877880,4.467742,434,10.06
New Customers,519128.69,1522.371525,3.662757,5.351906,341,7.91
Promising,279369.35,802.785489,39.525862,2.974138,348,8.07
About to Sleep,337515.88,641.665171,196.627376,2.110266,526,12.20
Need Atention,155947.70,363.514452,60.165501,1.289044,429,9.95
At Risk,93824.70,298.804777,19.449045,1.512739,314,7.28


## SQL Lite

In [32]:
dp = transactions_per_customer.copy()
dp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4312 entries, 12347 to 18287
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GrossRevenue   4312 non-null   float64
 1   Recency        4312 non-null   int64  
 2   Frequency      4312 non-null   int64  
 3   Products       4312 non-null   int64  
 4   Items          4312 non-null   int64  
 5   Country        4312 non-null   object 
 6   AvarageTicket  4312 non-null   float64
dtypes: float64(2), int64(4), object(1)
memory usage: 269.5+ KB


In [33]:
#endpoint = 'sqlite:///insiders_db.sqlite' #local
endpoint = 'sqlite:////Users/Alysson/Documents/Projects/E-Commerce-Clusterization/scr/sqlite/insiders_db.sqlite' #local


#C:\Users\Alysson\Documents\Projects\E-Commerce-Clusterization\scr\models
#postgre
#endpoint = f'postgresql://{pg_user}:{pg_passwd}@{pg_host}:{pg_port}'
#C:\Users\Alysson\Documents\Projects\E-Commerce-Clusterization\notebooks]

db = create_engine(endpoint, poolclass=NullPool)
conn = db.connect()

In [34]:
#check if table exists on sqlite
check_table = """
     SELECT name FROM sqlite_master WHERE type='table' AND name='insiders';
 """
df_check = pd.read_sql_query(check_table, conn)

#0 = table does not exist, 1 = table exists
if len(df_check) == 0:  
    query_create_table_insiders = """
        CREATE TABLE insiders (
            CustomerID               INTEGER,
            GrossRevenue             REAL,
            Recency                  INTEGER,
            Products                 INTEGER,
            Cluster                  INTEGER,
            LastTraining             TEXT        
           ) """


    conn = sqlite3.connect('insiders_db_sqlite')
    conn.execute( query_create_table_insiders )
    print('Table loyals was created!')
else:
    print('Table loyals exists!')

Table loyals exists!


In [35]:
db = create_engine(endpoint, poolclass=NullPool)
dp.to_sql('insiders', con=db, if_exists='append', index=False )#index=False to ignore dataframe index

In [36]:
#consult database
query = """
    SELECT * FROM insiders
"""
df = pd.read_sql_query(query, db)
df

,GrossRevenue,Recency,Frequency,Products,Items,Country,AvarageTicket
0,4310.00,3,7,182,2458,Iceland,161.68
1,1437.24,76,4,23,2332,Finland,308.64
2,1457.55,19,1,72,630,Italy,20.24
3,294.40,311,1,16,196,Norway,18.40
4,1265.41,37,8,84,463,Norway,113.84
...,...,...,...,...,...,...,...
12931,180.60,278,1,10,45,United Kingdom,18.06
12932,80.82,181,1,7,54,United Kingdom,11.55
12933,176.60,8,3,13,98,United Kingdom,28.44
12934,2088.93,4,16,687,1395,United Kingdom,48.61


In [37]:
conn.close()